In [1]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
from PIL import Image    
train_on_gpu = torch.cuda.is_available()   


In [2]:
import pandas as pd
import os

df = pd.read_csv('flair-vgg16-data.csv', names=['_id', 'message', 'image_concept', 'published', 'disabled'])
df['available'] = 0

all_images_path = 'data/all_images'
for i, row in df.iterrows():
    if os.path.isfile(os.path.join(all_images_path, row['_id'] + '.jpg')):
        df.at[i, 'available']= 1    

df_published = df[(df.published == 1) & (df.available == 1) & df.message.notnull()]

df_disabled = df[(df.disabled == 1) & (df.available == 1) & df.message.notnull()]

print(f"published {len(df_published)}, disabled {len(df_disabled)}")

df_all = pd.concat([df_published, df_disabled], ignore_index=True)

df_all = df_all.reset_index(drop=True)

df_all        

published 2628, disabled 5348


,_id,message,image_concept,published,disabled,available
0,5e5836fee917e8d9a8a7b277,endless blues greatbarrierreef australia whits...,seascape water shoal sea turquoise sun tropica...,1,0,1
1,5e58343ded065ad79e312f3d,hamiltonisland,tree travel vacation seashore water hotel isla...,1,0,1
2,5e57dc939e88b6be2ac42800,we are going coconuts for hamiltonisland here ...,relaxation beach sea vacation sand recreation ...,1,0,1
3,5e55dca437fa5927dcdf02f3,en route to gbr embrace the elevation in luxur...,nature travel diving water sea underwater ocea...,1,0,1
4,5e55d69eb9e5b725cd7ba02f,golf course views hamiltonislandgolfcourse whi...,outdoors landscape beach sky nature rural nope...,1,0,1
...,...,...,...,...,...,...
7971,5e253779f1b8d48ba5de7d32,colours so bright they hurt your eyes tropical...,outdoors nature scenery landscape water land o...,0,1,1
7972,5e252d334610948976f731e5,호 주 학 생 비 자 치 료 마 사 지 과 정 치 료 마 사 지 과 정 은 마 사 ...,human person patient therapy massage heel spa,0,1,1
7973,5e252d334610948976f731e6,호 주 학 생 비 자 치 료 마 사 지 과 정 치 료 마 사 지 과 정 은 마 사 ...,plant paper text flower blossom,0,1,1
7974,5e252d3342307c89757703c0,호 주 학 생 비 자 치 료 마 사 지 과 정 치 료 마 사 지 과 정 은 마 사 ...,person human finger hand dating face arm,0,1,1


In [3]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df_all, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(val_df, test_size=0.4, random_state=42)

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print(f"train {len(train_df)}, val {len(val_df)}, test {len(test_df)}")

train 4785, val 1914, test 1277


In [4]:
from flair.embeddings import (
    Sentence, 
    WordEmbeddings, 
    FlairEmbeddings, 
    StackedEmbeddings, 
    DocumentRNNEmbeddings,
    BytePairEmbeddings
)
from flair.training_utils import store_embeddings
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
from torchvision import transforms, models
from PIL import Image
import os
import matplotlib.pyplot as plt

%matplotlib inline

def oversample_df(df):
    classes = ['published', 'disabled'] 
    classes_count = []
    for c in classes:    
        classes_count.append(len(df.loc[df[c] == 1]))
    
    max_count = max(classes_count)
    resample_ratios = [round(max_count/c) for c in classes_count]
            
    resampled = []
    for i in range(len(resample_ratios)):
        c = classes[i]
        ratio = resample_ratios[i]        
        for r in range(ratio):            
            resampled.append(df.loc[df[c] == 1])
            
    resampled_df = pd.concat(resampled, ignore_index=True)
    resampled_df = resampled_df.sample(frac=1)
    resampled_df = resampled_df.reset_index(drop=True)
    
    return resampled_df

resampled = oversample_df(train_df)
print(f"published {len(resampled.loc[resampled.published == 1])}")
print(f"disabled {len(resampled.loc[resampled.disabled == 1])}")


def balance_df(df):
    classes = ['published', 'disabled'] 
    classes_count = []
    for c in classes:    
        classes_count.append(len(df.loc[df[c] == 1]))
    
    min_count = min(classes_count)
    
    resampled = []
    for c in classes:
        resampled.append(df[df[c] == 1][:min_count])
        
    resampled_df = pd.concat(resampled, ignore_index=True)
    resampled_df = resampled_df.sample(frac=1)
    resampled_df = resampled_df.reset_index(drop=True)
    
    return resampled_df
    
    
resampled = balance_df(train_df)
print(f"published {len(resampled.loc[resampled.published == 1])}")
print(f"disabled {len(resampled.loc[resampled.disabled == 1])}")

def get_batches(df, transformer, oversample=False, batch_size=16):        
    if oversample:        
        df = oversample_df(df)
    else:
        df = balance_df(df)
        
    n_batches = len(df)//batch_size    
    for i in range(0, len(df), batch_size):
        sentences = [Sentence(txt) for txt in df[i:i+batch_size]['message']]
        labels = [1 if label else 0 for label in df[i:i+batch_size]['published']]
        
        images = []
        for _id in df[i:i+batch_size]['_id']:
            image = Image.open('data/all_images/'+_id+'.jpg').convert('RGB')
            images.append(transformer(image).unsqueeze(0))
            
        image_tensor = torch.cat(images, 0)
        
        yield sentences, image_tensor, torch.FloatTensor(labels)

    

/home/ec2-user/anaconda3/envs/efficientnet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ec2-user/anaconda3/envs/efficientnet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ec2-user/anaconda3/envs/efficientnet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ec2-user/anacond

published 3116
disabled 3227
published 1558
disabled 1558


In [5]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        # output size: (224 - 5)/1 + 1 = 220
        # output tensor: 32 x 220 x 220
        self.conv1 = nn.Conv2d(3, 32, 5)             
               
        # output size: 220/2 = 110
        # output matrix: 32 x 110 x 110
        self.pool1 = nn.MaxPool2d(2,2)
        
        # output size: (110 - 4)/1 + 1 = 107
        # output tensor: 64 x 107 x 107
        self.conv2 = nn.Conv2d(32, 64, 4)
        
        # output size: 107/2 = 53
        # output tensor: 64 x 53 x 53
        self.pool2 = nn.MaxPool2d(2,2)
        
        # output size: (53 - 3)/1 + 1 = 51
        # output tensor: 128 x 51 x 51
        self.conv3 = nn.Conv2d(64, 128, 3)
        
        # output size: 51/2 = 25
        # output tensor: 128 x 25 x 25
        self.pool3 = nn.MaxPool2d(2,2)
        
        # output size: (25 - 2)/1 + 1 = 24
        # output tensor: 256 x 24 x 24
        self.conv4 = nn.Conv2d(128, 256, 2)
        
        # output size: 24/2 = 12
        # output tensor: 256 x 12 x 12
        self.pool4 = nn.MaxPool2d(2,2)

        
        # output size: (12 - 1)/1 + 1 = 12
        # output tensor: 512 x 12 x 12
        self.conv5 = nn.Conv2d(256, 512, 1)
        
        # output size: 12/2 = 6
        # output tensor: 512 x 6 x 6
        self.pool5 = nn.MaxPool2d(2,2)

                               
        self.drop1 = nn.Dropout(p=0.1)
        self.drop2 = nn.Dropout(p=0.2)
        
        self.fc1 = nn.Linear(512*6*6, 1024)
        self.fc2 = nn.Linear(1024, 1)
        
        
        torch.nn.init.xavier_uniform(self.conv1.weight)
        torch.nn.init.xavier_uniform(self.conv2.weight)
        torch.nn.init.xavier_uniform(self.conv3.weight)
        torch.nn.init.xavier_uniform(self.conv4.weight)
        torch.nn.init.xavier_uniform(self.conv5.weight)
        torch.nn.init.xavier_uniform(self.fc1.weight)
        torch.nn.init.xavier_uniform(self.fc2.weight)
        
    def forward(self, sentences, x):
        batch_size = x.size(0)
        
        if train_on_gpu:
            x = x.cuda()
        
        x = self.drop1(self.pool1(F.relu(self.conv1(x))))
        
        x = self.drop1(self.pool2(F.relu(self.conv2(x))))
        
        x = self.drop1(self.pool3(F.relu(self.conv3(x))))
        
        x = self.drop1(self.pool4(F.relu(self.conv4(x))))
        
        x = self.drop1(self.pool5(F.relu(self.conv5(x))))
        
        x = x.view(batch_size, -1)
        x = F.relu(self.fc1(x))
        x = self.drop2(x)        
        x = self.fc2(x)
        return x        
    
    

model = MyModel()
if train_on_gpu:
    model = model.cuda()
print(model)


print(model(None, torch.zeros((1,3,224,224))).size())

/home/ec2-user/anaconda3/envs/efficientnet/lib/python3.6/site-packages/ipykernel_launcher.py:54: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/home/ec2-user/anaconda3/envs/efficientnet/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/home/ec2-user/anaconda3/envs/efficientnet/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/home/ec2-user/anaconda3/envs/efficientnet/lib/python3.6/site-packages/ipykernel_launcher.py:57: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/home/ec2-user/anaconda3/envs/efficientnet/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/home/ec2-user/anaconda3/envs/efficientnet/lib/python3.6/sit

MyModel(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(128, 256, kernel_size=(2, 2), stride=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1))
  (pool5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.1, inplace=False)
  (drop2): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=18432, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=1, bias=True)
)
torch.Size([1, 1])


In [6]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score



inv_normalize = transforms.Normalize(
   mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
   std=[1/0.229, 1/0.224, 1/0.225]
)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

image_dimension = 224
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(image_dimension),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    normalize
])        

test_transform = transforms.Compose([
    transforms.Resize((image_dimension, image_dimension)),
    transforms.ToTensor(),
    normalize    
])

    


from torch.optim import lr_scheduler

def train_model(model, epochs, lr, train_df, val_df, checkpoint_file, early_stopping=10):        
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.8)
    best_loss = np.inf
    no_improvement = 0

    if train_on_gpu:
        model = model.cuda()
        
    for epoch in range(epochs):        
        total_train_loss = 0
        total_val_loss = 0
        total_train = 0
        total_val = 0
        train_loss = 0
        val_loss = 0
        
        # Train
        model.train()        
        for i, (sentences, images, labels) in enumerate(get_batches(train_df, train_transform, True)):         
            if train_on_gpu:
                labels = labels.cuda()
            
            optimizer.zero_grad()
            
            out = model(sentences, images)
            loss = criterion(out.squeeze(), labels)
            loss.backward()
            
            #nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optimizer.step()
            
            total_train_loss += loss.item()
            total_train += labels.size(0)
            
            store_embeddings(sentences, 'cpu')
            
            if i % 10 == 0:
                print(f"Epoch {epoch}, Batch {i}, train loss {loss.item()/labels.size(0)}")
            
            
        train_loss = total_train_loss/total_train
        print(f"> Epoch {epoch}, train loss {train_loss}")
        
        
        # Eval
        model.eval()
        all_labels = np.array([])
        all_pred = np.array([])        
        for sentences, images, labels in get_batches(val_df, test_transform):
            if train_on_gpu:
                labels = labels.cuda()
            
            out = model(sentences, images)
            loss = criterion(out.squeeze(), labels)
            total_val_loss += loss.item()
            total_val += labels.size(0)
                                
            pred = torch.round(torch.sigmoid(out.squeeze()))
            
            # for matrix
            pred_np = pred.data.cpu().numpy() if train_on_gpu else pred.data.numpy()
            labels_np = labels.data.cpu().numpy() if train_on_gpu else labels.data.numpy()                    
            all_pred = np.concatenate([all_pred, pred_np])
            all_labels = np.concatenate([all_labels, labels_np])
                        
            # clear memory
            store_embeddings(sentences, 'cpu')
            
            
        val_loss = total_val_loss / total_val
        f1 = f1_score(all_labels, all_pred, average='weighted')
        acc = accuracy_score(all_labels, all_pred)
        
        print(f"> Epoch {epoch}, val loss {val_loss}, accuracy {acc}, f1_score {f1}")
        
        if val_loss < best_loss:
            best_loss = val_loss
            no_improvement = 0
            torch.save(model.state_dict(), checkpoint_file)
            print("Saved model.")
        else:
            no_improvement += 1
            print("No improvement.")
            if no_improvement >= early_stopping:
                print(f"Early Stopping")
                break
                
                
        # reduce learning rate
        scheduler.step()

            
                                              
checkpoint_file = 'custom_cnn_model.pt'      
lr = 0.001
epochs = 50         

train_model(model, epochs, lr, train_df, val_df, checkpoint_file)


Epoch 0, Batch 0, train loss 0.044128403067588806
Epoch 0, Batch 10, train loss 0.045231640338897705
Epoch 0, Batch 20, train loss 0.04155183956027031
Epoch 0, Batch 30, train loss 0.04200037941336632
Epoch 0, Batch 40, train loss 0.04160936176776886
Epoch 0, Batch 50, train loss 0.03597522899508476
Epoch 0, Batch 60, train loss 0.049866754561662674
Epoch 0, Batch 70, train loss 0.04419554024934769
Epoch 0, Batch 80, train loss 0.041537266224622726
Epoch 0, Batch 90, train loss 0.0436345636844635
Epoch 0, Batch 100, train loss 0.04237906262278557
Epoch 0, Batch 110, train loss 0.04124557226896286
Epoch 0, Batch 120, train loss 0.03947645425796509
Epoch 0, Batch 130, train loss 0.04014018550515175
Epoch 0, Batch 140, train loss 0.04337479919195175
Epoch 0, Batch 150, train loss 0.04203856736421585
Epoch 0, Batch 160, train loss 0.04278353974223137
Epoch 0, Batch 170, train loss 0.039997123181819916
Epoch 0, Batch 180, train loss 0.044882938265800476
Epoch 0, Batch 190, train loss 0.0422

Epoch 3, Batch 320, train loss 0.03607296198606491
Epoch 3, Batch 330, train loss 0.03665335476398468
Epoch 3, Batch 340, train loss 0.04982275143265724
Epoch 3, Batch 350, train loss 0.04847220331430435
Epoch 3, Batch 360, train loss 0.040311574935913086
Epoch 3, Batch 370, train loss 0.04558607563376427
Epoch 3, Batch 380, train loss 0.04325726628303528
Epoch 3, Batch 390, train loss 0.04032911732792854
> Epoch 3, train loss 0.041687841551096755
> Epoch 3, val loss 0.042583943568458855, accuracy 0.5976744186046512, f1_score 0.5950930485862941
Saved model.
Epoch 4, Batch 0, train loss 0.04030797630548477
Epoch 4, Batch 10, train loss 0.040858082473278046
Epoch 4, Batch 20, train loss 0.042447417974472046
Epoch 4, Batch 30, train loss 0.04611053690314293
Epoch 4, Batch 40, train loss 0.04414750635623932
Epoch 4, Batch 50, train loss 0.03949117287993431
Epoch 4, Batch 60, train loss 0.044487595558166504
Epoch 4, Batch 70, train loss 0.04399013891816139
Epoch 4, Batch 80, train loss 0.04

Epoch 7, Batch 210, train loss 0.04546167701482773
Epoch 7, Batch 220, train loss 0.03904763236641884
Epoch 7, Batch 230, train loss 0.03997502475976944
Epoch 7, Batch 240, train loss 0.04048467427492142
Epoch 7, Batch 250, train loss 0.03991119936108589
Epoch 7, Batch 260, train loss 0.039404552429914474
Epoch 7, Batch 270, train loss 0.04532230645418167
Epoch 7, Batch 280, train loss 0.04087156429886818
Epoch 7, Batch 290, train loss 0.04074954614043236
Epoch 7, Batch 300, train loss 0.04314976558089256
Epoch 7, Batch 310, train loss 0.040833987295627594
Epoch 7, Batch 320, train loss 0.0402429923415184
Epoch 7, Batch 330, train loss 0.04056434705853462
Epoch 7, Batch 340, train loss 0.033670179545879364
Epoch 7, Batch 350, train loss 0.04725867882370949
Epoch 7, Batch 360, train loss 0.04031580686569214
Epoch 7, Batch 370, train loss 0.045339856296777725
Epoch 7, Batch 380, train loss 0.04327091947197914
Epoch 7, Batch 390, train loss 0.037514932453632355
> Epoch 7, train loss 0.040

Epoch 11, Batch 90, train loss 0.0431549996137619
Epoch 11, Batch 100, train loss 0.046392254531383514
Epoch 11, Batch 110, train loss 0.03976084291934967
Epoch 11, Batch 120, train loss 0.042561084032058716
Epoch 11, Batch 130, train loss 0.04505384713411331
Epoch 11, Batch 140, train loss 0.03992954641580582
Epoch 11, Batch 150, train loss 0.0401659831404686
Epoch 11, Batch 160, train loss 0.036695703864097595
Epoch 11, Batch 170, train loss 0.043377723544836044
Epoch 11, Batch 180, train loss 0.03415998816490173
Epoch 11, Batch 190, train loss 0.05077485367655754
Epoch 11, Batch 200, train loss 0.04095795005559921
Epoch 11, Batch 210, train loss 0.041418932378292084
Epoch 11, Batch 220, train loss 0.043778009712696075
Epoch 11, Batch 230, train loss 0.03911814093589783
Epoch 11, Batch 240, train loss 0.0319441482424736
Epoch 11, Batch 250, train loss 0.04329684376716614
Epoch 11, Batch 260, train loss 0.030780859291553497
Epoch 11, Batch 270, train loss 0.04750007390975952
Epoch 11,

Epoch 14, Batch 380, train loss 0.04482386261224747
Epoch 14, Batch 390, train loss 0.03922957554459572
> Epoch 14, train loss 0.03877400716697119
> Epoch 14, val loss 0.0394467027381409, accuracy 0.6596899224806202, f1_score 0.6584731007839361
Saved model.
Epoch 15, Batch 0, train loss 0.03839360922574997
Epoch 15, Batch 10, train loss 0.04142832010984421
Epoch 15, Batch 20, train loss 0.03975613787770271
Epoch 15, Batch 30, train loss 0.04663298279047012
Epoch 15, Batch 40, train loss 0.03539949655532837
Epoch 15, Batch 50, train loss 0.03825460374355316
Epoch 15, Batch 60, train loss 0.0406566858291626
Epoch 15, Batch 70, train loss 0.04677731543779373
Epoch 15, Batch 80, train loss 0.038029931485652924
Epoch 15, Batch 90, train loss 0.03449748456478119
Epoch 15, Batch 100, train loss 0.040652282536029816
Epoch 15, Batch 110, train loss 0.0330142006278038
Epoch 15, Batch 120, train loss 0.03748701512813568
Epoch 15, Batch 130, train loss 0.041344430297613144
Epoch 15, Batch 140, tra

...

Epoch 25, Batch 390, train loss 0.03789776563644409
> Epoch 25, train loss 0.03643814356530406
> Epoch 25, val loss 0.03943733656129172, accuracy 0.6565891472868217, f1_score 0.6465886750306583
No improvement.
Epoch 26, Batch 0, train loss 0.044968754053115845
Epoch 26, Batch 10, train loss 0.03319312632083893
Epoch 26, Batch 20, train loss 0.04738796874880791
Epoch 26, Batch 30, train loss 0.03554488345980644
Epoch 26, Batch 40, train loss 0.035676341503858566
Epoch 26, Batch 50, train loss 0.029145393520593643
Epoch 26, Batch 60, train loss 0.02851555496454239
Epoch 26, Batch 70, train loss 0.04006858542561531
Epoch 26, Batch 80, train loss 0.032019615173339844
Epoch 26, Batch 90, train loss 0.03689422458410263
Epoch 26, Batch 100, train loss 0.046456821262836456
Epoch 26, Batch 110, train loss 0.038001418113708496
Epoch 26, Batch 120, train loss 0.03300659358501434
Epoch 26, Batch 130, train loss 0.029569555073976517
Epoch 26, Batch 140, train loss 0.05561809241771698
Epoch 26, Batc

Epoch 29, Batch 250, train loss 0.04096140339970589
Epoch 29, Batch 260, train loss 0.020912256091833115
Epoch 29, Batch 270, train loss 0.0337306372821331
Epoch 29, Batch 280, train loss 0.033207863569259644
Epoch 29, Batch 290, train loss 0.028332248330116272
Epoch 29, Batch 300, train loss 0.029454713687300682
Epoch 29, Batch 310, train loss 0.03014862723648548
Epoch 29, Batch 320, train loss 0.03538237139582634
Epoch 29, Batch 330, train loss 0.03928515315055847
Epoch 29, Batch 340, train loss 0.02776039019227028
Epoch 29, Batch 350, train loss 0.044649526476860046
Epoch 29, Batch 360, train loss 0.03497619181871414
Epoch 29, Batch 370, train loss 0.028214674443006516
Epoch 29, Batch 380, train loss 0.026746641844511032
Epoch 29, Batch 390, train loss 0.03978221118450165
> Epoch 29, train loss 0.03584456285918415
> Epoch 29, val loss 0.0379096145084662, accuracy 0.672093023255814, f1_score 0.6720833676371698
Saved model.
Epoch 30, Batch 0, train loss 0.042283184826374054
Epoch 30, 

Epoch 33, Batch 110, train loss 0.02177116647362709
Epoch 33, Batch 120, train loss 0.026389770209789276
Epoch 33, Batch 130, train loss 0.044450677931308746
Epoch 33, Batch 140, train loss 0.032670099288225174
Epoch 33, Batch 150, train loss 0.030066397041082382
Epoch 33, Batch 160, train loss 0.02912496216595173
Epoch 33, Batch 170, train loss 0.032931625843048096
Epoch 33, Batch 180, train loss 0.03567907214164734
Epoch 33, Batch 190, train loss 0.04458506405353546
Epoch 33, Batch 200, train loss 0.04976477473974228
Epoch 33, Batch 210, train loss 0.03296174854040146
Epoch 33, Batch 220, train loss 0.030750341713428497
Epoch 33, Batch 230, train loss 0.031294096261262894
Epoch 33, Batch 240, train loss 0.04964863136410713
Epoch 33, Batch 250, train loss 0.04246348887681961
Epoch 33, Batch 260, train loss 0.044054239988327026
Epoch 33, Batch 270, train loss 0.03057599999010563
Epoch 33, Batch 280, train loss 0.026494566351175308
Epoch 33, Batch 290, train loss 0.01528957486152649
Epo

> Epoch 36, train loss 0.034077902284120325
> Epoch 36, val loss 0.03762450356816137, accuracy 0.6775193798449612, f1_score 0.6762110709941499
No improvement.
Epoch 37, Batch 0, train loss 0.03291543200612068
Epoch 37, Batch 10, train loss 0.02767348475754261
Epoch 37, Batch 20, train loss 0.041218049824237823
Epoch 37, Batch 30, train loss 0.05423139035701752
Epoch 37, Batch 40, train loss 0.03221574425697327
Epoch 37, Batch 50, train loss 0.02579713985323906
Epoch 37, Batch 60, train loss 0.03440096229314804
Epoch 37, Batch 70, train loss 0.02793767675757408
Epoch 37, Batch 80, train loss 0.03201704099774361
Epoch 37, Batch 90, train loss 0.03541402518749237
Epoch 37, Batch 100, train loss 0.04714319109916687
Epoch 37, Batch 110, train loss 0.025888849049806595
Epoch 37, Batch 120, train loss 0.03369162976741791
Epoch 37, Batch 130, train loss 0.03283318877220154
Epoch 37, Batch 140, train loss 0.039214592427015305
Epoch 37, Batch 150, train loss 0.037331823259592056
Epoch 37, Batch 

Epoch 40, Batch 260, train loss 0.023086046800017357
Epoch 40, Batch 270, train loss 0.022053591907024384
Epoch 40, Batch 280, train loss 0.01770361140370369
Epoch 40, Batch 290, train loss 0.03303427994251251
Epoch 40, Batch 300, train loss 0.030379768460989
Epoch 40, Batch 310, train loss 0.031134292483329773
Epoch 40, Batch 320, train loss 0.028637312352657318
Epoch 40, Batch 330, train loss 0.046447426080703735
Epoch 40, Batch 340, train loss 0.027898458763957024
Epoch 40, Batch 350, train loss 0.012829294428229332
Epoch 40, Batch 360, train loss 0.02809789963066578
Epoch 40, Batch 370, train loss 0.03349566459655762
Epoch 40, Batch 380, train loss 0.02170201949775219
Epoch 40, Batch 390, train loss 0.039883650839328766
> Epoch 40, train loss 0.032971125114289214
> Epoch 40, val loss 0.037667813758517424, accuracy 0.6906976744186046, f1_score 0.6884735013941857
No improvement.
Early Stopping


In [7]:
def eval_model(model, test_df):
    if train_on_gpu:
        model = model.cuda()
        
    model.eval()
    num_correct = 0
    num_total = 0
    all_pred = np.array([])
    all_labels = np.array([])
    for i, (sentences, images, labels) in enumerate(get_batches(test_df, test_transform)):
        if train_on_gpu:
            labels = labels.cuda()
            
        out = model(sentences, images)
        pred = torch.round(torch.sigmoid(out.squeeze()))

        correct = (pred == labels)
        correct = correct.cpu().numpy() if train_on_gpu else correct.numpy()        
        num_correct += np.sum(correct)
        num_total += labels.size(0)


        
        pred_np = pred.data.cpu().numpy() if train_on_gpu else pred.data.numpy()
        labels_np = labels.data.cpu().numpy() if train_on_gpu else labels.data.numpy()                    
        all_pred = np.concatenate([all_pred, pred_np])
        all_labels = np.concatenate([all_labels, labels_np])
        
        
        store_embeddings(sentences, 'cpu')

    
    f1 = f1_score(all_labels, all_pred, average='weighted')
    acc = accuracy_score(all_labels, all_pred)
    
    print(f"{num_correct}/{num_total} correct. Accuracy: {num_correct*100/num_total} %, acc {acc}, f1 {f1}")
    
    
best_model = MyModel()
best_model.load_state_dict(torch.load(checkpoint_file))
eval_model(best_model, test_df)


/home/ec2-user/anaconda3/envs/efficientnet/lib/python3.6/site-packages/ipykernel_launcher.py:54: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/home/ec2-user/anaconda3/envs/efficientnet/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/home/ec2-user/anaconda3/envs/efficientnet/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/home/ec2-user/anaconda3/envs/efficientnet/lib/python3.6/site-packages/ipykernel_launcher.py:57: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/home/ec2-user/anaconda3/envs/efficientnet/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/home/ec2-user/anaconda3/envs/efficientnet/lib/python3.6/sit

578/850 correct. Accuracy: 68.0 %, acc 0.68, f1 0.6797003147581681
